# Import

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import time
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchsummary import summary
from thop import profile

%matplotlib inline
from matplotlib import pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True

# Rotate and Flip Function

In [ ]:
char = 'abcdefghijklmnopqrs'
def rotate90(move_list):
    char_RowtoCol = 'abcdefghijklmnopqrs'
    char_ColtoRow = 'srqponmlkjihgfedcba'
    return_list = []
    for chess in move_list:
        return_list.append(chess[:2] + char_ColtoRow[char.find(chess[3])] + char_RowtoCol[char.find(chess[2])] + chess[4:])
    return return_list

def rotate180(move_list):
    char_RowtoRow = 'srqponmlkjihgfedcba'
    char_ColtoCol = 'srqponmlkjihgfedcba'
    return_list = []
    for chess in move_list:
        return_list.append(chess[:2] + char_RowtoRow[char.find(chess[2])] + char_ColtoCol[char.find(chess[3])] + chess[4:])
    return return_list

def rotate270(move_list):
    char_ColtoRow = 'abcdefghijklmnopqrs'
    char_RowtoCol = 'srqponmlkjihgfedcba'
    return_list = []
    for chess in move_list:
        return_list.append(chess[:2] + char_ColtoRow[char.find(chess[3])] + char_RowtoCol[char.find(chess[2])] + chess[4:])
    return return_list

def flip_up_down(move_list):
    char_RowtoRow = 'srqponmlkjihgfedcba'
    return_list = []
    for chess in move_list:
        return_list.append(chess[:2] + char_RowtoRow[char.find(chess[2])] + chess[3:])
    return return_list

def flip_rotate(move_list, type):
    function = [rotate90, rotate180, rotate270, flip_up_down]
    if type % 4 != 0:
        move_list = function[type % 4 - 1](move_list)
    if type // 4 != 0:
        move_list = function[3](move_list)
    return move_list

# Make DataLoader

In [ ]:
def Read_CSV(type):
    df = open('./29_Training Dataset/Training Dataset/kyu_train.csv').read().splitlines()
    color = [line.strip().split(',')[1] for line in df]
    games = [i.split(',', 2)[-1] for i in df]
    x = []
    y = []
    prediction_color = []

    for idx, game in enumerate(tqdm(games)):
        moves = game.split(',')
        moves = flip_rotate(moves, type)
        if color[idx] == 'B':
            for count in range(2, len(moves), 2):
                prediction_color.append(color[idx])
                x.append((moves[:count]))
                y.append((moves[count]))
        else:
            for count in range(1, len(moves), 2):
                prediction_color.append(color[idx])
                x.append((moves[:count]))
                y.append((moves[count]))
    print("Total data: ", len(y))
    return x, y, prediction_color

In [ ]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
dirR = [-1, -1, -1, 0, 0, 1, 1, 1, -2, -2, -2, -2, -2, -1, -1, 0, 0, 1, 1, 2, 2 ,2 ,2 ,2, -3, -3, -3, -3, -3, -3, -3
       , -2, -2, -1, -1, 0, 0, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3]
dirC = [-1, 0, 1, -1, 1, -1, 0, 1, -2, -1, 0, 1, 2, -2, 2, -2, 2, -2, 2, -2, -1, 0, 1, 2, -3, -2, -1, 0, 1, 2, 3, -3
       , 3, -3, 3, -3, 3, -3, 3, -3, 3, -3, -2, -1, 0, 1, 2, 3]


def prepare_input(moves, color):
    x = np.zeros((19,19,21))
    
    lstn = 5
    if(len(moves) < lstn):
        lstn = len(moves)
    
    for i in range(1,lstn+1):
        column = coordinates[moves[-i][2]]
        row = coordinates[moves[-i][3]]
        x[row, column, 21 - i] = 1
        
    
    for move in moves:
        colors = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if colors == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if colors == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
            
    my_piece = x[:,:,0]
    opt_piece = x[:,:,1]
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[::, 0:-1] = my_piece[::, 1:]
    tmp_my_piece[::, [-1]] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[::, 1:] = my_piece[::, 0:-1]
    tmp_my_piece[::, [0]] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[0:-1, ::] = my_piece[1:, ::]
    tmp_my_piece[[-1], ::] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_my_piece = np.empty((19,19))
    tmp_my_piece[1:, ::] = my_piece[0:-1, ::]
    tmp_my_piece[[0], ::] = 0
    x[:,:,4] += tmp_my_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[::, 0:-1] = opt_piece[::, 1:]
    tmp_opt_piece[::, [-1]] = 0
    x[:,:,9] += tmp_opt_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[::, 1:] = opt_piece[::, 0:-1]
    tmp_opt_piece[::, [0]] = 0
    x[:,:,9] += tmp_opt_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[0:-1, ::] = opt_piece[1:, ::]
    tmp_opt_piece[[-1], ::] = 0
    x[:,:,9] += tmp_opt_piece
    
    tmp_opt_piece = np.empty((19,19))
    tmp_opt_piece[1:, ::] = opt_piece[0:-1, ::]
    tmp_opt_piece[[0], ::] = 0
    x[:,:,9] += tmp_opt_piece
                        
    x[:,:,3] = np.where(x[:,:,2] == 1, 0, 1)    
        
    x[:,:,5] = np.where(x[:,:,4] == 1, 1, 0)
    x[:,:,6] = np.where(x[:,:,4] == 2, 1, 0)
    x[:,:,7] = np.where(x[:,:,4] == 3, 1, 0)
    x[:,:,8] = np.where(x[:,:,4] == 4, 1, 0)
    x[:,:,4] = np.where(x[:,:,4] == 0, 1, 0)
   
    x[:,:,10] = np.where(x[:,:,9] == 1, 1, 0)
    x[:,:,11] = np.where(x[:,:,9] == 2, 1, 0)
    x[:,:,12] = np.where(x[:,:,9] == 3, 1, 0)
    x[:,:,13] = np.where(x[:,:,9] == 4, 1, 0)
    x[:,:,9] = np.where(x[:,:,9] == 0, 1, 0)
        
    x[:,:,15] = np.where(x[:,:,14] == 0, 1, 0)    
        
    x = np.transpose(x, (2, 0, 1))
    if color == 'W':
        x[[0, 1], :, :] = x[[1, 0], :, :]
        x[[4, 9], :, :] = x[[9, 4], :, :]
        x[[5, 10], :, :] = x[[10, 5], :, :]
        x[[6, 11], :, :] = x[[11, 6], :, :]
        x[[7, 12], :, :] = x[[12, 7], :, :]
        x[[8, 13], :, :] = x[[13, 8], :, :]
    return x
    #x[0, :, :]放要預測的棋子, x[1, :, :]放對手的棋子, x[2, :, :]放有棋子的地方

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    index = column*19+row
    return torch.tensor(index, dtype=torch.long)

def ToImage(game):
    game = np.array(game)
    plt.imshow(game, cmap='gray')
    plt.show()

In [ ]:
class game(Dataset):
    def __init__(self, input, label, color):
        self.input = input
        self.label = label
        self.color = color
    def __len__(self):
        return len(self.input)
    def __getitem__(self,idx):
        moves = self.input[idx]
        color = self.color[idx]
        input = torch.from_numpy(np.float32(prepare_input(self.input[idx], self.color[idx])))
        label = prepare_label(self.label[idx])
        return {'input': input, 'label': label}

In [ ]:
def make_dataloader(type):
    x, y, prediction_color = Read_CSV(type)
    dataset = game(x, y, prediction_color)
    train_ds, val_ds = torch.utils.data.random_split(dataset, [int(len(y)*0.9), len(y) - int(len(y)*0.9)])
    train_dl = DataLoader(train_ds, 32, shuffle=True)
    val_dl = DataLoader(val_ds, 32, shuffle=False)
    return train_dl, val_dl, train_ds, val_ds, x, y, prediction_color

# Residual Networks

In [ ]:
class InputBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(InputBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=5, stride=1, padding=2, bias=False)
    
    def forward(self, x):
        out = self.conv1(x) + self.conv2(x)
        out = F.relu(out)
        return out

class SqueezeExcitationBlock(nn.Module):
    def __init__(self, in_planes, ratio):
        super(SqueezeExcitationBlock, self).__init__()
        self.dense1 = nn.Linear(in_planes, in_planes // ratio, bias=False)
        self.dense2 = nn.Linear(in_planes // ratio, in_planes, bias=False)
        
    def forward(self, x):
        se = F.adaptive_avg_pool2d((x), (1, 1))
        se = se.reshape((se.shape[0], -1))
        se = F.relu(self.dense1(se))
        se = F.sigmoid(self.dense2(se))     
        x = x * se.unsqueeze(dim=-1).unsqueeze(dim=-1)
        return x
    
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, ratio=16):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.seblock = SqueezeExcitationBlock(out_planes, ratio)
        self.shortcut = nn.Sequential()
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        out = self.seblock(out)
        return out
        
class ResNet(nn.Module):
    def __init__(self, in_planes=4, num_blocks=20, num_classes=361, ratio=16):
        super(ResNet, self).__init__()
        self.layer1 = InputBlock(in_planes, out_planes=128, stride=1)
        self.layer2 = self.make_layer(BasicBlock, 128, num_blocks, ratio)
        self.layer3 = nn.Conv2d(128, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.flatten = nn.Flatten()
    
    def make_layer(self, block, planes, num_blocks, ratio):
        layers = []
        for n in range(num_blocks):
            layers.append(block(planes, planes, 1, ratio))
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.flatten(out)
        return out

In [ ]:
model = ResNet(in_planes=21, num_blocks=20, num_classes=361).to('cuda')

# Testing Model Computational Cost

In [ ]:
input = torch.rand(1, 21, 19, 19).cuda()

model.eval()
out = model(input)

summary(model, input_size=(21, 19, 19))
print(f'From input shape: {input.shape} to output shape: {out.shape}')


macs, parm = profile(model, inputs=(input, ))
print(f'FLOPS: {macs * 2 / 1e9} G, Params: {parm / 1e6} M.')

# Optimizer、Scheduler、Criterion

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000025)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.99, patience=40000)

# Training

In [ ]:
def calculus_accuracy_top1(pro, label):
    prediction = pro.argmax(dim=1)
    correct_prediction = (prediction == label).float()
    accuracy = torch.mean(correct_prediction) * 100
    return accuracy.item()

def calculus_accuracy_top5(pro, label):
    top5_indices = pro.topk(5, dim=1)[1]
    accuracy = 0
    for i in range(5):
        prediction = top5_indices[:, i].unsqueeze(1)
        correct_prediction = (prediction == label.unsqueeze(1)).float()
        accuracy += torch.mean(correct_prediction) * 100
    return accuracy.item()

In [ ]:
def Test(model, device, val_dl):
    model.eval()
    val_loss = 0
    acc_top1 = 0
    acc_top5 = 0
    with torch.no_grad():
        for idx, data in enumerate(tqdm(val_dl)):
            input = data['input'].to(device)
            label = data['label'].to(device)
            out = model(input)
            val_loss += criterion(out, label)
            acc_top1 += calculus_accuracy_top1(out, label)
            acc_top5 += calculus_accuracy_top5(out, label)

    val_loss /= len(val_dl)
    acc_top1 /= len(val_dl)
    acc_top5 /= len(val_dl)
    print('\nTest Set Average loss: {:.10f}'.format(val_loss))
    logging.info('Test Set Average loss: {:.10f}'.format(val_loss))
    print('The top1 accuracy of model in validation data is {:.4f}%'.format(acc_top1))
    logging.info('The top1 accuracy of model in validation data is {:.4f}%'.format(acc_top1))
    print('The top5 accuracy of model in validation data is {:.4f}%'.format(acc_top5))
    logging.info('The top5 accuracy of model in validation data is {:.4f}%'.format(acc_top5))
    return val_loss.item()

In [ ]:
save_path = './Kyu_Adam_Weight'
if not os.path.isdir(save_path):
    os.mkdir(save_path)


lrs = []
train = []
val = []
def Train(epochs, model, device):
    best_loss = float('inf')
    logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG,
                       handlers=[logging.FileHandler("Kyu_Adam.txt"),logging.StreamHandler()])
    logging.info(f'Start Training at {time.asctime()}')
    logging.info('='*60)
    for epoch in range(1, epochs + 1):
        print('Prepare DataLoader...')
        train_dl, val_dl, train_ds, val_ds, x, y, prediction_color = make_dataloader(epoch-1)
        model.train()
        epoch_loss = 0.
        for batch_idx, data in enumerate(tqdm(train_dl)):
            input = data['input'].to(device)
            label = data['label'].to(device)
            lrs.append(optimizer.param_groups[0]['lr'])
            optimizer.zero_grad()
            out = model(input)
            loss = criterion(out, label)
            loss.backward()
            optimizer.step()
            scheduler.step(loss)
            epoch_loss += loss.item()
        print('Train Epoch: {} Loss: {:.10f}'.format(epoch, epoch_loss/len(train_dl)))
        logging.info('Train Epoch: {} Loss: {:.10f}'.format(epoch, epoch_loss/len(train_dl)))
        train_loss = epoch_loss/len(train_dl)
        train.append(train_loss)
        val_loss = Test(model, device, val_dl)
        val.append(val_loss)
        
        if val_loss < best_loss:
            best_loss = val_loss
            print("Saving checkpoint...")
            state = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'best_loss': best_loss }
            torch.save(state, './checkpoint_kyu_adam.pth')

        state = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'best_loss': best_loss }
        torch.save(state, f'{save_path}/checkpoint_Kyu_Adam_{epoch}.pth')
        logging.info(f'This epoch is end {time.asctime()}')
        logging.info('='*60)
            
        del train_dl, val_dl, train_ds, val_ds, x, y, prediction_color
    print(f'Best loss in this training process : {best_loss}')

In [ ]:
Train(13, model, 'cuda')

# Plot

In [ ]:
def plot_lrs(lrs):
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.')

plot_lrs(lrs)

In [ ]:
def plot_losses(train, val):
    plt.plot(train, '-bx')
    plt.plot(val, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')

plot_losses(train, val)

# Load Model

In [ ]:
weight_path = './checkpoint_kyu_adam.pth'
checkpoint = torch.load(weight_path)
model.load_state_dict(checkpoint['model_state_dict'], strict=True)

# Testing

In [ ]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argsort(prediction)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

chartonumbers = {k:v for k,v in enumerate(chars)}

In [ ]:
df = open('./29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/kyu_test_public.csv').read().splitlines()
color = [line.strip().split(',')[1] for line in df]
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for idx, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list, color[idx]))

with open('./upload_Kyu_Adam.csv', 'a+') as f:
    for idx, test_data in enumerate(tqdm(x_testing)):
        test_data = torch.from_numpy(np.array(test_data)).float().to('cuda')
        test_data = test_data.reshape(1, test_data.shape[0], 19, 19)
        predictions = model(test_data).to('cpu').detach().numpy()
        prediction_chars = top_5_preds_with_chars(predictions)
        answer_row = games_id[idx] + ',' + ','.join(prediction_chars[0]) + '\n'
        f.write(answer_row)

# Private Data

In [ ]:
df = open('./29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/kyu_test_private.csv').read().splitlines()
color = [line.strip().split(',')[1] for line in df]
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for idx, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list, color[idx]))

with open('./upload_Kyu_Adam.csv', 'a+') as f:
    for idx, test_data in enumerate(tqdm(x_testing)):
        
        test_data = torch.from_numpy(np.array(test_data)).float().to('cuda')
        test_data = test_data.reshape(1, test_data.shape[0], 19, 19)
        predictions = model(test_data).to('cpu').detach().numpy()
        prediction_chars = top_5_preds_with_chars(predictions)
        answer_row = games_id[idx] + ',' + ','.join(prediction_chars[0]) + '\n'
        f.write(answer_row)